# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [20]:
# Import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [21]:
# Getting data
orders = pd.read_csv('orders.csv')
display(orders.head(), orders.shape)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


(397924, 14)

In [22]:
#Deleting unnecessary row 'Unnamed:0'
orders = orders.drop(['Unnamed: 0'], axis=1)

In [23]:
# Checking for null values
orders.isna().sum()

InvoiceNo       0
StockCode       0
year            0
month           0
day             0
hour            0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
CustomerID      0
Country         0
amount_spent    0
dtype: int64

In [24]:
#Checking column types

orders.dtypes

InvoiceNo         int64
StockCode        object
year              int64
month             int64
day               int64
hour              int64
Description      object
Quantity          int64
InvoiceDate      object
UnitPrice       float64
CustomerID        int64
Country          object
amount_spent    float64
dtype: object

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

#### Sub Problem 1: How to aggregate the amount_spent for unique customers?

In [25]:
# Group by' Customer_ID', 'Country' and aggregate sum of'amount_spent'

cust_tamount = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent':sum})
display(cust_tamount.head(), cust_tamount.shape)


,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40


(4347, 1)

#### Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

In [26]:
# Getting data from any quantile
quantiles = cust_tamount['amount_spent'].quantile([0.75,0.95])
quantiles

0.75    1661.195
0.95    5774.278
Name: amount_spent, dtype: float64

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

In [27]:
# Labels
labels=['General Customers','Preferred Customers','VIP Customers']

# Classificating customers according to their total amount quantile
cust_tamount['customer_type'] = pd.cut(cust_tamount['amount_spent'], bins=[-np.inf, quantiles[0.75], quantiles[0.95], np.inf], labels=labels)
cust_tamount

,,amount_spent,customer_type
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP Customers
12347,Iceland,4310.00,Preferred Customers
12348,Finland,1797.24,Preferred Customers
12349,Italy,1757.55,Preferred Customers
12350,Norway,334.40,General Customers
...,...,...,...
18280,United Kingdom,180.60,General Customers
18281,United Kingdom,80.82,General Customers
18282,United Kingdom,178.05,General Customers


CustomerID  Country       
12346       United Kingdom     True
12347       Iceland           False
12348       Finland           False
12349       Italy             False
12350       Norway            False
                              ...  
18280       United Kingdom    False
18281       United Kingdom    False
18282       United Kingdom    False
18283       United Kingdom    False
18287       United Kingdom    False
Name: customer_type, Length: 4347, dtype: bool

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [30]:
#Identifying VIP / Preferred customers
vip_customer = cust_tamount[cust_tamount['customer_type']=='VIP Customers']
preferred_customer = cust_tamount[cust_tamount['customer_type']=='Preferred Customers']

display('Vip count: ', vip_customer.nunique(), 'Preferred count: ', preferred_customer.nunique())

'Vip count: '

amount_spent     218
customer_type      1
dtype: int64

'Preferred count: '

amount_spent     868
customer_type      1
dtype: int64

In [18]:
cust_tamount.head()

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40


In [ ]:
# Looking for VIP for each country

cust_tamount.groupby(['Country']).agg({})

,amount_spent
Country,
United Kingdom,3423635.81
Netherlands,280206.02
EIRE,261204.69
Australia,124914.53
France,106383.09
Germany,106369.06
Sweden,31906.82
Japan,28406.43
Switzerland,26315.86


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
#Assigning VIP customers to country
vip_country = vip_customer.groupby(['Country']).agg({'amount_spent':sum}).sort_values('amount_spent', ascending=False)
vip_country

In [12]:
#creating preferred list

preferred_country = preferred_customer.groupby(['Country']).agg({'amount_spent':sum}).sort_values('amount_spent', ascending=False)

#Combining both


vip_and_preferred = pd.merge(left = vip_country,
                                 right = preferred_country,
                                 how = 'inner', 
                                 left_on = "Country", 
                                 right_on= "Country")

vip_and_preferred

,amount_spent_x,amount_spent_y
Country,,
United Kingdom,3423635.81,2204199.671
EIRE,261204.69,4341.210
Australia,124914.53,10056.290
France,106383.09,68311.580
Germany,106369.06,87709.480
Sweden,31906.82,2484.980
Japan,28406.43,6822.690
Switzerland,26315.86,19066.130
Spain,25391.20,21173.500


In [14]:
#Adding a sumatory column to show the raking

vip_and_preferred['total_amount_spent'] = vip_and_preferred['amount_spent_x'] + vip_and_preferred['amount_spent_y']
vip_and_preferred

,amount_spent_x,amount_spent_y,total_amount_spent
Country,,,
United Kingdom,3423635.81,2204199.671,5627835.481
EIRE,261204.69,4341.210,265545.900
Australia,124914.53,10056.290,134970.820
France,106383.09,68311.580,174694.670
Germany,106369.06,87709.480,194078.540
Sweden,31906.82,2484.980,34391.800
Japan,28406.43,6822.690,35229.120
Switzerland,26315.86,19066.130,45381.990
Spain,25391.20,21173.500,46564.700
